In [0]:
%%shell
if ! command -v julia 3>&1 > /dev/null
then
    wget 'https://julialang-s3.julialang.org/bin/linux/x64/1.4/julia-1.4.1-linux-x86_64.tar.gz' -O julia.tar.gz
    tar -x -f julia.tar.gz -C /usr/local --strip-components 1
    rm julia.tar.gz
fi
    julia -e '\
        using Pkg                                                          ;\
        Pkg.update()                                                       ;\
        Pkg.add("IJulia")                                                  ;\
        Pkg.add("BenchmarkTools")                                          ;\
        Pkg.add("CuArrays")                                                ;\
        Pkg.add("GLM")                                                     ;\
        Pkg.add("StatsBase")                                               ;\
        Pkg.add("MLDataUtils")                                             ;\       
        Pkg.add("Statistics")                                              ;\
        Pkg.add("DataFrames")                                              ;\
        Pkg.add("CSV")                                                     ;\
        Pkg.add("RDatasets")                                               ;\
        Pkg.add("Gadfly")                                                  ;\
        Pkg.add("Cairo")                                                   ;\
        Pkg.add("Fontconfig")                                              ;\
        Pkg.update()                                                       ;\
        Pkg.precompile()                                                   ;\'
echo 'Done'

# Julia 機器學習：GLM 線性迴歸

## 作業 027：波士頓房價預測資料集

請使用 GLM 中的模型，建立一個預測模型來預測波士頓的房價。
https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

The name for this dataset is simply boston. It has two prototasks: nox, in which the nitrous oxide level is to be predicted; and price, in which the median value of a home is to be predicted

      Crim - per capita crime rate by town

      Zn - proportion of residential land zoned for lots over 25,000 sq.ft.

      Indus - proportion of non-retail business acres per town.

      Chas - Charles River dummy variable (1 if tract bounds river; 0 otherwise)

      NOx - nitric oxides concentration (parts per 10 million)

      Rm - average number of rooms per dwelling

      Age - proportion of owner-occupied units built prior to 1940

      Dis - weighted distances to five Boston employment centres

      Rad - index of accessibility to radial highways

      Tax - full-value property-tax rate per $10,000

      PTRatio - pupil-teacher ratio by town

      Black - 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town

      LStat - % lower status of the population

      MedV - Median value of owner-occupied homes in $1000's


Variable #14 seems to be censored at 50.00 (corresponding to a median price of $50,000); 

Censoring is suggested by the fact that the highest median price of exactly $50,000 is reported in 16 cases, 

while 15 cases have prices between $40,000 and $50,000, with prices rounded to the nearest hundred. Harrison and Rubinfeld do not mention any censoring.



In [0]:
!julia -e '\
using GLM, RDatasets, MLDataUtils, Statistics                                                                                      ;\
data = dataset("MASS", "Boston")                                                                                                   ;\
println(  first(data, 1) )                                                                                                         ;\
println(   )                                                                                                                       ;\
println( describe(data) )                                                                                                          ;\
indecies = MLDataUtils.shuffleobs(collect(1:nrow(data)))                                                                           ;\
train_ind, test_ind = MLDataUtils.splitobs(indecies, at=0.999)                                                                     ;\
train = data[train_ind, :]                                                                                                         ;\
test = data[test_ind, :]                                                                                                           ;\
logreg = GLM.lm(@formula(MedV ~ Crim + Zn + Indus + Chas + NOx + Rm  + Age + Dis + Rad + Tax + PTRatio + Black + LStat ), train)   ;\
pred = predict(logreg, test)                                                                                                       ;\
println( "test", test )                                                                                                            ;\
println( "pred", pred )                                                                                                            ;\'